In [1]:
import torch
from torch.utils.data import TensorDataset
import numpy as np

train_dataset = torch.load('/data0/ps/wucunqi/homework/Aeolus-Analysis/data/Aeolus/Flight_chain/chain_data_2024/flight_chain_test_2024.pt')
test_dataset = torch.load('/data0/ps/wucunqi/homework/Aeolus-Analysis/data/Aeolus/Flight_chain/chain_data_2024/flight_chain_test_2024.pt')
valid_dataset = torch.load('/data0/ps/wucunqi/homework/Aeolus-Analysis/data/Aeolus/Flight_chain/chain_data_2024/flight_chain_val_2024.pt')


/tmp/ipykernel_124064/2604676879.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load('/data0/ps/wucunqi/homework/Aeolus-Analysis/data/Aeolus/Fligh

In [2]:
from torch.utils.data import DataLoader

dataloader = DataLoader(train_dataset, batch_size=80000, shuffle=True, num_workers=20)
test_dataloader = DataLoader(test_dataset, batch_size=80000, shuffle=True, num_workers=20)
    

In [3]:
from tqdm import tqdm
progress_bar = tqdm(dataloader, desc=f'Epoch {0+1}/{100}')

for batch in progress_bar:
    dense_feat, sparse_feat, two_labels, valid_lens = [x.to("cuda") for x in batch]
    print(dense_feat.shape, sparse_feat.shape, two_labels.shape, valid_lens.shape)

Epoch 1/100: 100%|██████████| 12/12 [00:02<00:00,  4.11it/s]

torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]) torch.Size([80000, 6]) torch.Size([80000])
torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]) torch.Size([80000, 6]) torch.Size([80000])
torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]) torch.Size([80000, 6]) torch.Size([80000])
torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]) torch.Size([80000, 6]) torch.Size([80000])
torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]) torch.Size([80000, 6]) torch.Size([80000])
torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]) torch.Size([80000, 6]) torch.Size([80000])
torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]) torch.Size([80000, 6]) torch.Size([80000])
torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]) torch.Size([80000, 6]) torch.Size([80000])
torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]) torch.Size([80000, 6]) torch.Size([80000])
torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]) torch.Size([80000, 6]) torch.Size([80000])
torch.Size([80000, 6, 7]) torch.Size([80000, 6, 8]

In [4]:
# 读取embedding信息
 
import pickle
 
f = open('/data0/ps/wucunqi/homework/Aeolus-Analysis/new_feature_columns.pkl', 'rb')
feature_columns = pickle.load(f)
f.close()
print(feature_columns)

(['O_TEMP', 'D_TEMP', 'O_PRCP', 'D_PRCP', 'O_WSPD', 'D_WSPD', 'FLIGHTS'], [{'feat_name': 'MONTH', 'feat_num': 12, 'embed_dim': 7}, {'feat_name': 'DAY_OF_WEEK', 'feat_num': 7, 'embed_dim': 7}, {'feat_name': 'CRS_ARR_TIME_HOUR', 'feat_num': 24, 'embed_dim': 7}, {'feat_name': 'CRS_DEP_TIME_HOUR', 'feat_num': 24, 'embed_dim': 7}, {'feat_name': 'ORIGIN_INDEX', 'feat_num': 322, 'embed_dim': 7}, {'feat_name': 'DEST_INDEX', 'feat_num': 322, 'embed_dim': 7}, {'feat_name': 'OP_CARRIER', 'feat_num': 15, 'embed_dim': 7}, {'feat_name': 'OP_CARRIER_FL_NUM', 'feat_num': 6854, 'embed_dim': 7}])


In [5]:
feature_columns

(['O_TEMP', 'D_TEMP', 'O_PRCP', 'D_PRCP', 'O_WSPD', 'D_WSPD', 'FLIGHTS'],
 [{'feat_name': 'MONTH', 'feat_num': 12, 'embed_dim': 7},
  {'feat_name': 'DAY_OF_WEEK', 'feat_num': 7, 'embed_dim': 7},
  {'feat_name': 'CRS_ARR_TIME_HOUR', 'feat_num': 24, 'embed_dim': 7},
  {'feat_name': 'CRS_DEP_TIME_HOUR', 'feat_num': 24, 'embed_dim': 7},
  {'feat_name': 'ORIGIN_INDEX', 'feat_num': 322, 'embed_dim': 7},
  {'feat_name': 'DEST_INDEX', 'feat_num': 322, 'embed_dim': 7},
  {'feat_name': 'OP_CARRIER', 'feat_num': 15, 'embed_dim': 7},
  {'feat_name': 'OP_CARRIER_FL_NUM', 'feat_num': 6854, 'embed_dim': 7}])

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print('使用设备:', device)

使用设备: cuda


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
dense_feature_cols, sparse_feature_cols = feature_columns

In [9]:
import torch
import torch.nn as nn

class FlightDelayCNN_LSTM(nn.Module):
    def __init__(self, feature_columns, hidden_size, num_layers, output_size):
        super(FlightDelayCNN_LSTM, self).__init__()
        
        self.dense_feature_cols, self.sparse_feature_cols = feature_columns
        self.embed_layers = nn.ModuleDict({
            'embed_'+str(i):nn.Embedding(feat['feat_num'], feat['embed_dim']) 
            for i, feat in enumerate(self.sparse_feature_cols)
        })
        
        self.input_dim = len(self.dense_feature_cols) + len(self.sparse_feature_cols)*self.sparse_feature_cols[0]['embed_dim']
        self.cnn = nn.Sequential(
            nn.Conv1d(self.input_dim, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm1d(64),
           # nn.MaxPool1d(kernel_size=2, stride=1)
        )

        self.lstm = nn.LSTM(
            input_size=64,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=0.2 if num_layers > 1 else 0
        )
        
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        dense_inputs = x[:, :, :len(self.dense_feature_cols)]
        sparse_inputs = x[:, :, len(self.dense_feature_cols):].long()
        
        sparse_embeds = [self.embed_layers[f'embed_{i}'](sparse_inputs[:,:,i]) 
                        for i in range(len(self.sparse_feature_cols))]
        sparse_embeds = torch.cat(sparse_embeds, dim=-1)
        
        x = torch.cat([sparse_embeds, dense_inputs], dim=-1)  
        
        x = x.permute(0, 2, 1)  
        x = self.cnn(x)         
        x = x.permute(0, 2, 1)  
        
        x, _ = self.lstm(x)
        x = self.fc(x)
        
        return x

In [10]:
hidden_size = 32  
num_layers = 2  
output_size = 2  

model = FlightDelayCNN_LSTM(feature_columns, hidden_size, num_layers, output_size)
model = model.to(device)

In [11]:
model

FlightDelayCNN_LSTM(
  (embed_layers): ModuleDict(
    (embed_0): Embedding(12, 7)
    (embed_1): Embedding(7, 7)
    (embed_2): Embedding(24, 7)
    (embed_3): Embedding(24, 7)
    (embed_4): Embedding(322, 7)
    (embed_5): Embedding(322, 7)
    (embed_6): Embedding(15, 7)
    (embed_7): Embedding(6854, 7)
  )
  (cnn): Sequential(
    (0): Conv1d(63, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (lstm): LSTM(64, 32, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=32, out_features=2, bias=True)
)

In [12]:
def sequence_mask(X, valid_len, value=0):
    #在序列中屏蔽不相关的项
    maxlen = X.size(1)
    mask = torch.arange((maxlen), dtype=torch.float32,
                        device=X.device)[None, :] < valid_len[:, None]
    X[~mask] = value
    return X

X = torch.tensor([[1, 2, 3, 4], [4, 5, 6, 9]])
sequence_mask(X, torch.tensor([1, 3]))

tensor([[1, 0, 0, 0],
        [4, 5, 6, 0]])

In [13]:
class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    
    def __init__(self, pos_weight=4.5):
        super(MaskedSoftmaxCELoss, self).__init__(reduction='none')
        self.pos_weight = pos_weight

    def forward(self, pred, label, valid_len):
        # 带权重张量，初始值为1
        weights = torch.ones_like(label, dtype=torch.float)
        
        # 增加正类的权重
        weights[label == 1] *= self.pos_weight
        
        # 只取序列的有效部分
        weights = sequence_mask(weights, valid_len)
        
        # 有效加权损失
        unweighted_loss = super().forward(pred.permute(0, 2, 1), label)
        weighted_loss = unweighted_loss * weights
        
        total_loss = weighted_loss.sum()
        total_valid_len = valid_len.sum()
       # print(unweighted_loss, weighted_loss, total_loss, total_valid_len)
        
        
        if total_valid_len > 0:
            return total_loss / total_valid_len
        else:
            return torch.tensor(0.0).to(weighted_loss.device)

In [ ]:
import torch
import os
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


optimizer = torch.optim.Adam(
    model.parameters(), 
    lr=0.001,          
    weight_decay=1e-4
)

criterion = MaskedSoftmaxCELoss()  

CHECKPOINT_PATH = "cnnlstm_model_checkpoint.pth"
BEST_MODEL_PATH = "cnnlstm_best_model.pth"

num_epochs = 50
best_val_loss = float('inf')
loss_history = []
lr_history = []

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=num_epochs,  
    eta_min=1e-6       
)


if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
    model.load_state_dict(checkpoint['model_state'])
    optimizer.load_state_dict(checkpoint['optimizer_state'])
    scheduler.load_state_dict(checkpoint['scheduler_state'])  
    start_epoch = checkpoint['epoch'] + 1
    best_val_loss = checkpoint['best_val_loss']
    loss_history = checkpoint['train_loss_history']
    lr_history = checkpoint['lr_history']  
    print(f"恢复训练：从第{start_epoch}轮开始，最佳验证损失：{best_val_loss:.4f}")
else:
    start_epoch = 0

for epoch in range(start_epoch, num_epochs):
    model.train()
    epoch_loss = 0.0
    progress_bar = tqdm(dataloader, desc=f'Epoch {epoch+1}/{num_epochs}')
    
    for batch in progress_bar:
        dense_feat, sparse_feat, two_labels, valid_lens = [x.to(device) for x in batch]
        optimizer.zero_grad()
        # two_labels: 可能是 [B, T] 或 [B, T, 2]，这里统一取成 [B, T]
        labels = two_labels[:, :, 0] if two_labels.dim() == 3 else two_labels
        
        
        with torch.cuda.amp.autocast():
            features = torch.cat([dense_feat, sparse_feat], dim=-1)
            outputs = model(features)
            loss = criterion(outputs, labels.long(), valid_lens)
        
        loss.backward()
        optimizer.step()
        
        loss_history.append(loss.item())
        epoch_loss += loss.item()
        avg_loss = epoch_loss / (progress_bar.n + 1)
        progress_bar.set_postfix({
            'train_loss': f"{avg_loss:.4f}",
            'lr': f"{optimizer.param_groups[0]['lr']:.2e}"
        })

    scheduler.step()  
    current_lr = optimizer.param_groups[0]['lr']
    lr_history.append(current_lr)
    print(f"\n当前学习率：{current_lr:.2e}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for i, val_batch in enumerate(test_dataloader):
            if i >= 10000:  
                break
            v_dense, v_sparse, two_v_labels, v_lens = [x.to(device) for x in val_batch]
            v_labels = two_v_labels[:, :, 0] if two_v_labels.dim() == 3 else two_v_labels
            v_features = torch.cat([v_dense, v_sparse], dim=-1)
            v_outputs = model(v_features)
            val_loss += criterion(v_outputs, v_labels.long(), v_lens).item()
    
    avg_val_loss = val_loss / min(10000, len(test_dataloader))
    print(f"验证损失：{avg_val_loss:.4f} | 历史最佳：{best_val_loss:.4f}")

    checkpoint = {
        'epoch': epoch,
        'model_state': model.state_dict(),
        'optimizer_state': optimizer.state_dict(),
        'scheduler_state': scheduler.state_dict(), 
        'best_val_loss': best_val_loss,
        'train_loss_history': loss_history,
        'lr_history': lr_history  
    }
    torch.save(checkpoint, CHECKPOINT_PATH)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        print(f"★ 发现新最佳模型，验证损失：{best_val_loss:.4f}")

# 训练结束后，保存最后一次 checkpoint（原来的 final_checkpoint 变量未定义会报错）
torch.save(checkpoint, "cnnlstm_final_checkpoint.pth")
print(f"训练完成！最佳模型已保存至 {BEST_MODEL_PATH}")

Epoch 1/50:   0%|          | 0/12 [00:02<?, ?it/s]


IndexError: too many indices for tensor of dimension 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({
    'font.family': 'Times New Roman',  
    'font.size': 12,                  
    'axes.labelsize': 14,              
    'axes.linewidth': 1.2,             
    'lines.linewidth': 2,              
    'xtick.labelsize': 12,             
    'ytick.labelsize': 12,             
    'mathtext.fontset': 'stix',       
    'savefig.dpi': 300,                
    'figure.figsize': (8, 5)          
})

epochs = np.arange(len(lr_history))  
max_lr = max(lr_history)
min_lr = min(lr_history)


fig, ax = plt.subplots()


ax.plot(epochs, lr_history, 
        color='#1f77b4',  
        marker='o', 
        markersize=5,
        markevery=int(len(epochs)/10),  
        label='Learning Rate')


ax.annotate(f'Initial LR: {max_lr:.1e}', 
           xy=(0, max_lr), 
           xytext=(5, max_lr*1.2),
           arrowprops=dict(arrowstyle="->", lw=1.5))

ax.annotate(f'Min LR: {min_lr:.1e}', 
           xy=(epochs[-1], min_lr), 
           xytext=(epochs[-1]-20, min_lr*1.5),
           arrowprops=dict(arrowstyle="->", lw=1.5))


ax.set_xlabel('Training Epochs', fontweight='bold')
ax.set_ylabel('Learning Rate', fontweight='bold')
ax.set_title('Cosine Annealing Learning Rate Schedule\n(T_max=120 epochs)', 
            fontsize=14, pad=15)
ax.grid(True, linestyle='--', alpha=0.6)
ax.set_yscale('log')  


secax = ax.secondary_xaxis('top')
secax.set_xticks(np.linspace(0, len(epochs), 5))
secax.set_xticklabels(['0%', '25%', '50%', '75%', '100%'])
secax.set_xlabel('Training Progress', fontweight='bold')


ax.legend(frameon=True, 
         loc='upper right',
         facecolor='white',
         framealpha=0.8)


plt.tight_layout()
plt.savefig('cosine_lr_schedule.pdf', format='pdf', bbox_inches='tight')
#plt.close()

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt


mpl.rcParams.update({
    'font.family': 'serif',               
    'font.size': 12,                      
    'axes.labelsize': 14,                 
    'axes.linewidth': 1.5,                
    'xtick.direction': 'in',              
    'ytick.direction': 'in',              
    'xtick.labelsize': 12,                
    'ytick.labelsize': 12,                
    'grid.linestyle': '--',              
    'grid.alpha': 0.6,                    
    'mathtext.fontset': 'stix',           
    'savefig.dpi': 300,                   
    'savefig.bbox': 'tight'               
})


fig, ax = plt.subplots(figsize=(8, 5))   
ax.plot(
    range(1, len(loss_history)+1), 
    loss_history, 
    marker='o', 
    markersize=6,
    linewidth=2,
    color='#1f77b4',                      
    markeredgecolor='k',                  
    markeredgewidth=0.5
)


ax.set_xlabel('Epoch', fontweight='bold') 
ax.set_ylabel('Loss', fontweight='bold')
ax.set_title('Training Loss Progression', fontsize=16, pad=15)  
ax.grid(True, which='both', alpha=0.6)    


ax.xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))  
ax.yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(2))  
ax.tick_params(which='minor', length=3, width=1)           


plt.savefig('training_loss.pdf', dpi=300)
plt.show()

In [ ]:
def load_flight_model(device, 
                     feature_columns,
                     hidden_size,
                     num_layers,
                     output_size,
                     model_path=BEST_MODEL_PATH):

    
    model = FlightDelayCNN_LSTM(
        feature_columns=feature_columns,
        hidden_size=hidden_size,
        num_layers=num_layers,
        output_size=output_size
    ).to(device)
    
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from sklearn.metrics import (accuracy_score, recall_score, precision_score, 
                           f1_score, roc_auc_score, roc_curve, confusion_matrix)
from torch.utils.data import DataLoader

def evaluate_model(model, dataloader, device):
    model.eval()  
    all_logits, all_labels, all_valid_lens = [], [], []

    with torch.no_grad():
        for batch in dataloader:
            dense_feat, sparse_feat, two_labels, valid_lens = [x.to(device) for x in batch]
            labels = two_labels[:, :, 0] if two_labels.dim() == 3 else two_labels
            feature = torch.cat([dense_feat, sparse_feat], dim=-1)
            logits = model(feature)
            
            all_logits.append(logits.cpu())
            all_labels.append(labels.cpu())
            all_valid_lens.append(valid_lens.cpu())

    logits = torch.cat(all_logits, dim=0).numpy()
    labels = torch.cat(all_labels, dim=0).numpy()
    valid_lens = torch.cat(all_valid_lens, dim=0).numpy()

    position_mask = (np.arange(logits.shape[1]) < valid_lens[:, None])
    
    valid_probs = torch.softmax(torch.from_numpy(logits), dim=-1)[..., 1].numpy()
    final_probs = valid_probs[position_mask].flatten()
    final_labels = labels[position_mask].flatten()

    fpr, tpr, thresholds = roc_curve(final_labels, final_probs)
    J = tpr - fpr  
    optimal_idx = np.argmax(J)
    optimal_threshold = thresholds[optimal_idx]
    
    try:
        auc = roc_auc_score(final_labels, final_probs)
    except ValueError:
        auc = 0.5

    pred_labels = (final_probs >= optimal_threshold).astype(int)
    
    return {
        "Accuracy": accuracy_score(final_labels, pred_labels),
        "Precision": precision_score(final_labels, pred_labels, zero_division=0),
        "Recall": recall_score(final_labels, pred_labels, zero_division=0),
        "F1": f1_score(final_labels, pred_labels),
        "AUC": auc
    }, final_probs, final_labels

In [ ]:
BEST_MODEL_PATH

In [ ]:
model = load_flight_model(
    device=device,
    feature_columns=feature_columns,
    hidden_size=hidden_size,
    num_layers=num_layers,
    output_size=output_size
)
device = next(model.parameters()).device

train_dataloader = DataLoader(train_dataset, 
                            batch_size=80000, 
                            shuffle=False, num_workers=20)
val_dataloader = DataLoader(valid_dataset, 
                           batch_size=80000, 
                           shuffle=False, num_workers=20)
test_dataloader = DataLoader(test_dataset,  
                            batch_size=80000,
                            shuffle=False, num_workers=20)

train_metrics, _, _ = evaluate_model(model, train_dataloader, device)
val_metrics, _, _ = evaluate_model(model, val_dataloader, device)
test_metrics, final_probs, final_labels = evaluate_model(model, test_dataloader, device)  


print("\n{:<15} {:<10} {:<10} {:<10}".format("Metric", "Train", "Val", "Test"))
for key in train_metrics:
    print("{:<15} {:<10.4f} {:<10.4f} {:<10.4f}".format(
        key + ":", 
        train_metrics[key], 
        val_metrics[key],
        test_metrics[key]  
    ))